# Counting D combos

In each set of D(X,Y;Z,O), D(X,Z;Y,O) and D(Z,Y,X,O), or D1, D2, D3, there are three possible answers: >, < or =. How many unique combinations are there? 

In [19]:
import collections

myoptions=[">","<","="]
mytrees=["XYZ","XZY","ZYX"]
mypats={"XYZ":["XZ","YZ","OO"],"XZY":["XY","YZ","OO"],"ZYX":["XZ","XY","OO"]}
mysets=[]
for i in myoptions:
    for j in myoptions:
        for k in myoptions:
            mysets.append([i,j,k])
            
treenoadmix=(2,'OO')
treeadmix=(2,1)
similarrel=(1,1,'OO')
genpat=["21","1OO2","11OO1","111","OO3","2OO1"]
genpatnames=["treeadmix","slightconnect","similaradmix","allconnect","norelation","treenoadmix"]
uniqsets=collections.defaultdict(list)
for ind, i in enumerate(mysets):
    sisters = [ mypats[mytrees[treeind]][myoptions.index(equality)] for treeind,equality in enumerate(i) ]
    counts=collections.Counter(sisters)
    mystats=counts.most_common()
    treepat=''.join([ str(mystat[1]) for mystat in mystats if mystat[0] != 'OO' ])
    oos=[ mystat[0]+str(mystat[1]) for mystat in mystats if mystat[0] == 'OO' ]
    oos1=oos[0] if len(oos)>0 else ''
    uniqsets[treepat+oos1].append(''.join(i))
    #print ind,''.join(i), counts, treepat+oos1
    
dictpat={}
for i in uniqsets: 
    dictpat[genpatnames[genpat.index(i)]+"_"+i] = uniqsets[i]
print dictpat

{'slightconnect_1OO2': ['>==', '<==', '=>=', '=<=', '==>', '==<'], 'treeadmix_21': ['>>>', '>><', '><>', '<><', '<<>', '<<<'], 'allconnect_111': ['><<', '<>>'], 'similaradmix_11OO1': ['>>=', '><=', '>=<', '<>=', '<=>', '<=<', '=>>', '=<>', '=<<'], 'norelation_OO3': ['==='], 'treenoadmix_2OO1': ['>=>', '<<=', '=><']}


# 2. Real Data

In [15]:
import collections
pd="/mnt/solexa/mel_yang/2.2_neolithic/Dstats/frommpi/"
fh="UPA_all8"
tv="all"
pop1="EUPASNAMER"
indiv="Linzi"
out="Mbuti"
subset1="%s_%s_%s_%s" % (pop1,pop1,indiv,out)
subset2="%s_%s_%s_%s" % (indiv,pop1,pop1,out)
DorZind=7 ##Zind (5 is Dind)

myfile1=open(pd+"%s.D.%s.%s.log" % (fh,tv,subset1) )
myfile2=open(pd+"%s.D.%s.%s.log" % (fh,tv,subset2) )
results=[line.split() for line in myfile1 if line.split()[0]=="result:"]
results+=[line.split() for line in myfile2 if line.split()[0]=="result:"]
myfile1.close()
myfile2.close()

opptype={">":"<","<":">","=":"="}

pops={}
uniqueset=[]
for mylst in results:
    mypopns=mylst[1:4]
    val=float(mylst[DorZind])
    if val>=2.5:mytype=">"
    elif val<=-2.5: mytype="<"
    else: mytype="="
    checkind=[ind for ind,i in enumerate(uniqueset) if set(i)==set(mypopns)]
    if len(checkind)==0: 
        uniqueset.append(tuple(mypopns))
        x,y,z=tuple(mypopns)
    else: x,y,z=uniqueset[checkind[0]]

    if (x,y,z) not in pops: pops[(x,y,z)]=[0,0,0,0,0,0]
    
    if x==mypopns[0] and y==mypopns[1] and z==mypopns[2]: pops[(x,y,z)][0]=mytype
    elif x==mypopns[0] and y==mypopns[2] and z==mypopns[1]: pops[(x,y,z)][1]=mytype
    elif x==mypopns[2] and y==mypopns[1] and z==mypopns[0]: pops[(x,y,z)][2]=mytype
    elif x==mypopns[1] and y==mypopns[0] and z==mypopns[2]: pops[(x,y,z)][0]=opptype[mytype]
    elif x==mypopns[1] and y==mypopns[2] and z==mypopns[0]: pops[(x,y,z)][1]=opptype[mytype]
    elif x==mypopns[2] and y==mypopns[0] and z==mypopns[1]: pops[(x,y,z)][2]=opptype[mytype]
    else: print x,y,z; continue

        
        
genpat={'slightconnect_1OO2': ['>==', '<==', '=>=', '=<=', '==>', '==<'], 
       'treeadmix_21': ['>>>', '>><', '><>', '<><', '<<>', '<<<'], 
       'allconnect_111': ['><<', '<>>'], 
       'similaradmix_11OO1': ['>>=', '><=', '>=<', '<>=', '<=>', '<=<', '=>>', '=<>', '=<<'], 
       'norelation_OO3': ['==='], 
       'treenoadmix_2OO1': ['>=>', '<<=', '=><']}

mypatname=[]
mypats=[]
for i in genpat: 
    mypatname+=[i]*len(genpat[i])
    mypats+=genpat[i]
relationships=["X_02","X_12","X_01","X_12","X_02","X_01","02_01","01_02","02_12","01_12","12_02","12_01",
               "X_X_A","X_X_A","X_0","X_0","X_1","X_0","X_1","X_1","X_2","X_2","X_2","X_X_X","02_X","12_X","01_X"]

newsubset="Linzi_EUPASNAMER"
newfile=open(pd+"%s.D.%s.patterns.%s.txt" % (fh,tv,newsubset) ,'w')
newfile.write("X\tY\tZ\ttitle\tpattern\ttree\tadmix\n")
for ind,mypop in enumerate(pops): 
    mypattern = ''.join(pops[mypop][:3])
    mytitle=mypatname[mypats.index(mypattern)]
    myrel=relationships[mypats.index(mypattern)].split('_')
    if myrel[0]!="X": 
        other=[i for i in '012' if i not in myrel[0]][0]
        #print other
        tree="(%s,%s),%s" % (tuple([mypop[int(i)] for i in myrel[0]]+[mypop[int(other)]]))
    else: tree="NA"
        
    if myrel[1]!="X":
        if len(myrel[1]) == 2: admix="%s<->%s" % (tuple([mypop[int(i)] for i in myrel[1]]))
        else: 
            both=int(myrel[1][0])
            others=[i for i in [0,1,2] if i != both]
            admix="%s<->%s,%s<->%s" % (mypop[both],mypop[others[0]],mypop[both],mypop[others[1]])
    else: 
        if len(myrel)==2: admix="NA"
        else:
            if myrel[2]=="A": admix="ALL"
            elif myrel[2]=="X": admix="NONE"
    newfile.write('\t'.join(mypop)+'\t'+mytitle+'\t'+mypattern+'\t'+tree+'\t'+admix+'\n')

    #if ind==10: break
newfile.close()
        

In [79]:
genpat={'slightconnect_1OO2': ['>==', '<==', '=>=', '=<=', '==>', '==<'], 
       'treeadmix_21': ['>>>', '>><', '><>', '<><', '<<>', '<<<'], 
       'allconnect_111': ['><<', '<>>'], 
       'similaradmix_11OO1': ['>>=', '><=', '>=<', '<>=', '<=>', '<=<', '=>>', '=<>', '=<<'], 
       'norelation_OO3': ['==='], 
       'treenoadmix_2OO1': ['>=>', '<<=', '=><']}

mypatname=[]
mypats=[]
for i in genpat: 
    mypatname+=[i]*len(genpat[i])
    mypats+=genpat[i]
relationships=["X_02","X_12","X_01","X_12","X_02","X_01","02_01","01_02","02_12","01_12","12_02","12_01",
               "X_X_A","X_X_A","X_0","X_0","X_1","X_0","X_1","X_1","X_2","X_2","X_2","X_X_X","02_X","12_X","01_X"]


eas=["Oroqen","Daur","Hezhen","Xibo","Japanese","Korean",
        "Tu","Han","Tujia","Miao","Yi","She","Naxi","Atayal",
        "Ami","Lahu","Dai","Kinh","Burmese","Thai","Cambodian","Tibetan","Uygur"]
amer=["Clovis","Pima","Mayan","Mixe","Zapotec","Piapoco",
        "Karitiana","Surui","Quechua","Chane","Nahua",
        "Cree","Chipewyan",'Saqqaq']
sib=["Aleut","Tlingit","Mansi","Chukchi","Tubalar","Kyrgyz","Ulchi",
    "Mongola","Altaian","Even","Yakut","Itelman",
    "Eskimo_Sireniki","Eskimo_Naukan","Eskimo_Chaplin"]
pac=["Onge","Papuan","Maori","Igorot","Hawaiian",
         "Dusun","Bougainville","Australian"]
oas=["Sherpa","Kusunda"]
anc=["Tianyuan",'Longlin','Longlin_com','Xinyi',
        'Xinyi_other','Daxi','LiangDaoChineseNeolithic','Boisman_MN',
        'BoshanChineseNeolithic','Bbdong','Linzi','HDYM1','HQSDW',
        'Zongri','Pukagongma','Chokhopani1','Mebrak','Samdzong',
        'Kolyma_River',"Donghuigou","Balikun"]
eur=["UstIshim","Oase1","Kostenki14","GoyetQ116-1",
     "Vestonice16","Yana_old","Yana_old2",
     "Malta1",'AfontovaGora3',"ElMiron","Villabruna","Bichon",
     "Satsurblia","Kotias","Karelia","Motala12","Loschbour","LaBrana1",
     "Hungarian.KO1","Stuttgart","French","Sardinian","Saami"]
asn=eas+amer+sib+pac+oas+anc
popties=["(ASN,ASN),EUR","(EUR,EUR),ASN","(EAS,EAS),ASN-EAS","(AMER,AMER),ASN-AMER",
         "(EAS,EAS),ASN-EAS","(SIB,SIB),ASN-SIB","(OAS,OAS),ASN-OAS","(PAC,PAC),ASN-PAC"]
popchecks=[(asn,eur),(eur,asn),(eas,list(set(asn).difference(eas))),(amer,list(set(asn).difference(amer))),
           (eas,list(set(asn).difference(eas))),(sib,list(set(asn).difference(sib))),(oas,list(set(asn).difference(oas))),(pac,list(set(asn).difference(pac)))]


newsubset="Linzi_EUPASNAMER"
newfile=open(pd+"%s.D.%s.patterns.%s.txt" % (fh,tv,newsubset) ,'w')
newfile.write("X\tY\tZ\ttitle\tpattern\ttree\tadmix\trelationship\n")
for ind,mypop in enumerate(pops): 
    mypattern = ''.join(pops[mypop][:3])
    mytitle=mypatname[mypats.index(mypattern)]
    myrel=relationships[mypats.index(mypattern)].split('_')
    
    treeind=(-1,-1,-1)
    if myrel[0]!="X": 
        other=[i for i in '012' if i not in myrel[0]][0]
        #print other
        tree="(%s,%s),%s" % (tuple([mypop[int(i)] for i in myrel[0]]+[mypop[int(other)]]))
        treeind=tuple([int(i) for i in myrel[0]]+[int(other)])
    else: tree="NA"
        
    if myrel[1]!="X":
        if len(myrel[1]) == 2: admix="%s<->%s" % (tuple([mypop[int(i)] for i in myrel[1]]))
        else: 
            both=int(myrel[1][0])
            others=[i for i in [0,1,2] if i != both]
            admix="%s<->%s,%s<->%s" % (mypop[both],mypop[others[0]],mypop[both],mypop[others[1]])
    else: 
        if len(myrel)==2: admix="NA"
        else:
            if myrel[2]=="A": admix="ALL"
            elif myrel[2]=="X": admix="NONE"
                
    if treeind[0]!=-1:
        mypoprels = [ ind for ind,i in enumerate(popties) if mypop[treeind[0]] in popchecks[ind][0] and mypop[treeind[1]] in popchecks[ind][0] and mypop[treeind[2]] in popchecks[ind][1]] 
        if len(mypoprels)==0: mycategory="NA"
        else: mycategory=popties[mypoprels[0]]
    else: mycategory="NA"
        
    newfile.write('\t'.join(mypop)+'\t'+mytitle+'\t'+mypattern+'\t'+tree+'\t'+admix+'\t'+mycategory+'\n')

    #if ind==10: break
newfile.close()

In [77]:
list(set(asn).difference(eas))

['Even',
 'Kolyma_River',
 'Tianyuan',
 'Surui',
 'Papuan',
 'Tubalar',
 'Eskimo_Sireniki',
 'Clovis',
 'Zapotec',
 'Bougainville',
 'Kyrgyz',
 'BoshanChineseNeolithic',
 'Mebrak',
 'Tlingit',
 'Maori',
 'Kusunda',
 'Onge',
 'Karitiana',
 'Zongri',
 'Chipewyan',
 'Chokhopani1',
 'Mayan',
 'HDYM1',
 'Chukchi',
 'Linzi',
 'Boisman_MN',
 'Samdzong',
 'Mixe',
 'Eskimo_Naukan',
 'Ulchi',
 'Balikun',
 'Daxi',
 'Altaian',
 'Piapoco',
 'LiangDaoChineseNeolithic',
 'Saqqaq',
 'Pukagongma',
 'Mongola',
 'Sherpa',
 'Longlin',
 'Australian',
 'Eskimo_Chaplin',
 'Hawaiian',
 'Dusun',
 'Cree',
 'Longlin_com',
 'Xinyi',
 'Bbdong',
 'HQSDW',
 'Aleut',
 'Pima',
 'Chane',
 'Itelman',
 'Igorot',
 'Xinyi_other',
 'Nahua',
 'Yakut',
 'Donghuigou',
 'Quechua',
 'Mansi']